In [1]:
import os
import cv2
import torch
import pandas as pd
import copy
from skimage.util import random_noise
from torch.utils.data import Dataset
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import util
import easydict
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import datetime
from pathlib import Path
from PIL import Image

In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print('Device : {}'.format(device))

args = easydict.EasyDict({
    "device": device,
    "mnist_fn": './data/old_data/train.csv',
    "resume": None,

    "lr": 1e-4,
    "batch_size": 64,
    "epochs": 2000,
    "patient": 15,

    "dataset_ratio": 0.7,
})

util.seed_everything(777)
assert os.path.isfile(args.mnist_fn), 'wrong path'
# assert os.path.isdir(args.train_data_file_dir), 'wrong path'
# assert os.path.isdir(args.test_data_file_dir), 'wrong path'
# assert os.path.isfile(args.train_data_answer_dir), 'wrong path'
# assert os.path.isfile(args.submission_file_dir), 'wrong path'

Device : cuda:1


In [ ]:
class Mnist(Dataset):    
    def __init__(self, fn, transforms=None):        
        self.transforms = transforms
        df = pd.read_csv(fn)
        df = df.drop(['id', 'letter'], axis=1).values
        df = df.reshape(-1, 28, 28) / 255.0

        
        self.train_x = np.concatenate([img_gaussian, img_snp], axis=0)
        self.train_y = np.concatenate([img_original, img_original], axis=0)
        
    def __len__(self):
        return self.train_x.shape[0]
    
    def __getitem__(self, idx):
        if self.transforms:
            train_x = self.transforms(self.train_x[idx])
            train_y = self.transforms(self.train_y[idx])
        else:
            train_x = self.train_x[idx]
            train_y = self.train_y[idx]
             
        return train_x, train_y

In [11]:
df = pd.read_csv(args.mnist_fn)
df = df.drop(['id', 'digit'], axis=1)
letter = df['letter']
# df = df.reshape(-1, 28, 28) / 255.0
letter

0       L
1       B
2       L
3       D
4       A
       ..
2043    V
2044    L
2045    A
2046    Z
2047    Z
Name: letter, Length: 2048, dtype: object